### Libraries

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, DepthwiseConv2D, GlobalAveragePooling2D, Reshape, Activation, Multiply, Input, Add
from keras.optimizers import Adam

### Settings

In [2]:
### Paths ###

# IP102 path
ip_102_path = "../../../Data/MSiA 432/03_hw/IP102-Dataset/"

# FairFace path
fair_face_path = "../../../Data/MSiA 432/03_hw/FairFace/"

# Question 1

In word document attatched with this submission

# Question 2

In [3]:
columns = ["img_path", "target"]

train_df = pd.read_csv(ip_102_path + "train.txt", header=None, sep=" ", names=columns)
val_df = pd.read_csv(ip_102_path + "val.txt", header=None, sep=" ", names=columns)
test_df = pd.read_csv(ip_102_path + "test.txt", header=None, sep=" ", names=columns)

for df in [train_df, val_df, test_df]:
    df.target = df.target.astype(str)
    df['img_path'] = df['target'].astype(str) + '/' + df['img_path']

# Get # unique classes
num_classes = len(train_df.target.unique())

train_df.head()

,img_path,target
0,0/00002.jpg,0
1,0/00003.jpg,0
2,0/00005.jpg,0
3,0/00006.jpg,0
4,0/00008.jpg,0


In [4]:
# Define ImageDataGenerator for training (with augmentation)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest")

# Define ImageDataGenerator for validation and testing (without augmentation)
valid_test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
batch_size = 32
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=os.path.join(ip_102_path, "classification/train"),
    x_col="img_path",
    y_col="target",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)

val_generator = valid_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=os.path.join(ip_102_path, "classification/val"),
    x_col="img_path",
    y_col="target",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = valid_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=os.path.join(ip_102_path, "classification/test"),
    x_col="img_path",
    target_size=(100, 100),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

Found 45095 validated image filenames belonging to 102 classes.
Found 7508 validated image filenames belonging to 102 classes.
Found 22619 validated image filenames.


In [6]:
input_shape = (100, 100, 3)
optimizer = "adam"
loss = "categorical_crossentropy"
metrics = ["accuracy", tf.keras.metrics.AUC(name='auc')]

In [7]:
def create_model(input_shape, num_classes):

    model = Sequential([

        Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax'),

    ])


    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)  # Suitable for classification

    return model

model = create_model(input_shape, num_classes)

In [8]:
num_gpu = len(tf.config.list_physical_devices('GPU'))
print(f"{num_gpu} GPU(s) available")

1 GPU(s) available


In [9]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.n // val_generator.batch_size
)

Epoch 1/20
1409/1409 [==============================] - 307s 215ms/step - loss: 4.0218 - accuracy: 0.1044 - auc: 0.7629 - val_loss: 3.8215 - val_accuracy: 0.1319 - val_auc: 0.8033
Epoch 2/20
1409/1409 [==============================] - 230s 163ms/step - loss: 3.7598 - accuracy: 0.1457 - auc: 0.8092 - val_loss: 3.6224 - val_accuracy: 0.1595 - val_auc: 0.8310
Epoch 3/20
1409/1409 [==============================] - 227s 161ms/step - loss: 3.6471 - accuracy: 0.1611 - auc: 0.8253 - val_loss: 3.5305 - val_accuracy: 0.1812 - val_auc: 0.8368
Epoch 4/20
1409/1409 [==============================] - 228s 162ms/step - loss: 3.5565 - accuracy: 0.1765 - auc: 0.8376 - val_loss: 3.4680 - val_accuracy: 0.1978 - val_auc: 0.8449
Epoch 5/20
1409/1409 [==============================] - 233s 165ms/step - loss: 3.4941 - accuracy: 0.1877 - auc: 0.8448 - val_loss: 3.4549 - val_accuracy: 0.1927 - val_auc: 0.8467
Epoch 6/20
1409/1409 [==============================] - 223s 158ms/step - loss: 3.4292 - accuracy: 0

KeyboardInterrupt: 

# Question 3

# Question 4